In [2]:
import pandas as pd
import re
import numpy as np
from cytoolz import concat
from collections import Counter

In [5]:
df = pd.read_msgpack('https://bit.ly/2U1i0Lp')

In [6]:
df # comments taken from the consipiracy theory subreddit

,author,body,created_utc,id,link_id,retrieved_on,subreddit,created
23,AnonDidNothingWrong,Don't feed the trolls. Watch this.... it's lik...,2019-02-14 00:57:01,egfat4v,t3_aqdnp9,2019-02-14 00:57:02,conspiracy,2019-02-14 08:57:01
33,AnonDidNothingWrong,https://imgoat.com/uploads/5d8ce590ad/196567.jpg,2019-02-14 00:55:25,egfaov4,t3_aq4woc,2019-02-14 00:55:26,conspiracy,2019-02-14 08:55:25
43,AnonDidNothingWrong,Consider this: 1. https://imgoat.com/uploads/5...,2019-02-14 00:51:29,egfaedo,t3_apzili,2019-02-14 00:51:30,conspiracy,2019-02-14 08:51:29
67,seeking101,"i almost never put my hands to my face, i dont...",2019-02-14 00:46:23,egfa0i8,t3_aqal5w,2019-02-14 00:46:25,conspiracy,2019-02-14 08:46:23
76,seeking101,>the data needed to make a rational decision i...,2019-02-14 00:44:49,egf9wdo,t3_aqal5w,2019-02-14 00:44:50,conspiracy,2019-02-14 08:44:49
91,seeking101,that was by design no doubt. just like how if ...,2019-02-14 00:41:34,egf9np6,t3_aqal5w,2019-02-14 00:41:35,conspiracy,2019-02-14 08:41:34
94,seeking101,"just for the record, being ""anit flu shot"" doe...",2019-02-14 00:40:48,egf9lp9,t3_aqal5w,2019-02-14 00:40:49,conspiracy,2019-02-14 08:40:48
104,seeking101,>just like not exactly. one used adults and th...,2019-02-14 00:38:51,egf9gda,t3_aqbtc3,2019-02-14 00:38:52,conspiracy,2019-02-14 08:38:51
123,seeking101,"foot fetish is one of the most common, dont fe...",2019-02-14 00:35:38,egf97v0,t3_aqbtc3,2019-02-14 00:35:40,conspiracy,2019-02-14 08:35:38
160,AnonDidNothingWrong,SS Owen Benjamin provides convincing evidence ...,2019-02-14 00:27:32,egf8meg,t3_aqdnp9,2019-02-14 00:27:33,conspiracy,2019-02-14 08:27:32


In [7]:
df.loc[803]['body'] 

">90% is funded by FEDERAL tax dollars That's not true, some States pay less/more than others, though. It's actually very difficult to pinpoint the exact percentage of State/Federal funding. >which means Bezos uses it to get rich Well yea, he created a great company and employs half a million people. He deserves his wealth. >, does not pay for it, instead you and I pay for it They do pay for it, though. It's in other forms of taxes. This is what you and others, seem to not understand. You're either ignorant, or purposely spreading propaganda. So which is it? Furthermore, Amazon is one of the largest customers of USPS. Without Companies like Amazon and others, the postal service would fail. Sounds like you need a wee bit more logic and a shit tonne less reactionary emotion."

In [28]:
#'dollars' > 'FEDERAL tax dollars'
# FEDERAL: 'adjective'
# tax: 'noun'
# dollars: 'noun'
# ANN

import spacy
import en_core_web_sm #not necessary

In [29]:
nlp = spacy.load('en', disable=['parser','ner'])
# disabling part of pipelines

In [60]:
doc = nlp(df.loc[803]['body'])
doc[-2], doc[-2].pos_, doc[-2].lemma_, doc[-2].is_stop

170

In [32]:
def get_terms(doc): # to identify ANN followed by V
    terms = []
    start = -1 #start will be the token number where the sequence(Adj.) started. will be from -1, that we haven't seen yet.
    for i in range(len(doc)):
        if doc[i].pos_ in ['NOUN', 'ADJ']: #if it is N or adj,
            if start == -1: #have we already marked start of something?
                start = i
            # else: do nothing
        else: #if we are at the end? or at the start
            if start != -1 and i > start + 1 and doc[i-1].pos_ == 'NOUN' and not doc[i-1].is_stop: #and the preceding word is noun?
                terms.append(doc[start:i].lower_) #then we take the subpart and make it lowercase then added to terms
            start = -1
    #we also need to check if we are at the end of the sentence.
    if start != -1 and i > start + 1 and doc[i-1].is_stop:
        terms.append(doc[start:].lower_)
    return terms

get_terms(doc)

['exact percentage',
 'great company',
 'his wealth',
 'other forms',
 'largest customers',
 'postal service',
 'wee bit more logic',
 'shit tonne',
 'reactionary emotion']

In [34]:
%%time #printout the time taken

df['doc'] = list(nlp.pipe(df['body']))
df['terms'] = df['doc'].apply(get_terms)
freq = Counter(concat(df['terms']))
freq.most_common(10)

Wall time: 4min 44s


In [35]:
# dig down to individual users
top = df.groupby('author')['body'] \
        .count() \
        .sort_values(ascending=False)
top.head(10)

author
Putin_loves_cats       2530
AnonDidNothingWrong    2526
RMFN                   2181
rodental               1930
EdmondDantes777        1789
TheCIASellsDrugs       1575
NASAislyingtoyou       1486
Q_me_in                1445
mikeleoncraft          1435
RocketSurgeon22        1397
Name: body, dtype: int64

In [36]:
for name in top.head(10).index: #as we are interested in not numbers but who(index)
    print('---', name)
    subset = df['author'] == name
    freq = Counter(concat(df[subset]['terms'])) #take out the terms column, then make all comments into 1, then count frequency
    print(', '.join(t for t,f in freq.most_common(50))) #each of the term, frequency pairs, I will take only term
    print()

--- Putin_loves_cats
many people, my point, same thing, most people, my own country, first place, my opinion, submission statement, your soul, my mind, holy shit, small business, your comment, your post, conspiracy theory, dark priest class, welfare programs, long time, other way, fake news, my head, conspiracy theories, youtube channels, what people, bad thing, physical realm, obvious reasons, rabbit hole, many times, whole thing, my comment, full autos, minimum wage, other forms, good luck, our disinformation program, american public, military order, many years, your eyes, your mind, our society, reverse psychology, his actions, time travel, my mouth, your point, more sense, more power, my time

--- AnonDidNothingWrong
our country, text messages, white people, open borders, black people, citizen households, my point, my comment, same thing, plea deal, most people, average iq, black communities, their own race, deep state, your comment, many people, their children, fake news, tax cuts

In [61]:
for name in top.head(10).index:
    print('---',name)
    subset = df['author'] == name
    freq = pd.DataFrame({'all':
                          Counter(concat(df['terms'])),
                         'user':
                          Counter(concat(df[subset]['terms']))})
    freq['pmi'] = np.log2((freq['user'] * np.sum(freq['all'])) /
                          (freq['all'] * np.sum(freq['user'])) )
    print(' '.join(freq[freq['user']>5] 
                   .sort_values('pmi', ascending=False)
                   .head(10)
                   .index))
    print()

--- Putin_loves_cats
dark priest class my own country small business your soul holy shit my mind my point my opinion conspiracy theory same thing

--- AnonDidNothingWrong
% republican support citizen households their own race physical barrier non citizens native population ethnic diversity low awareness average hourly earnings black communities

--- RMFN
body autonomy strong species self defense self control his mother sandy hook trans people my place their religion free speech

--- rodental
cool story bud red team search bar bud symmetrical collapse blue team mod coup american government mental illness other hand free speech

--- EdmondDantes777
defensive coordinators my sweet summer child my nation mental hoops common people public record concentration camps constitutional republic authority figures political theater

--- TheCIASellsDrugs
opponent position maximum effect vote bots valid sources their emotional response steady stream other authorities other attack nuclear command site